<a href="https://colab.research.google.com/github/minisstreliya/binance/blob/main/binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install websocket-client

In [19]:
import websocket,  json
import time
import multiprocessing
import numpy as np

socket1 =  f'wss://stream.binance.com:9443/ws/btcusdt@kline_1m'
socket2 =  f'wss://stream.binance.com:9443/ws/ethusdt@kline_1m'
socket3 = f'wss://stream.binance.com:9443/ws/bnbbtc@kline_1m'


def on_message(ws, message):
  global marker
  global count
  
  data = json.loads(message)
  candle = data['k']
  close_price = float(candle['c'])
  if data['s'] == 'BTCUSDT':
    btcusdt.append(close_price)
  if data['s'] == 'BNBBTC':
    bnbbtc.append(close_price)
  if data['s'] == 'ETHUSDT':
    ethusdt.append(close_price)


  count += 1  
  if count > marker:
    count = 1
    ws.close()


def on_close(ws, close_status_code, close_msg):
  print("Connection closed")


def running1(btcusdt, ethusdt, bnbbtc):
  ws = websocket.WebSocketApp(socket1, on_message=on_message, on_close=on_close)
  ws.run_forever()


def running2(btcusdt, ethusdt, bnbbtc):
  ws = websocket.WebSocketApp(socket2, on_message=on_message, on_close=on_close)
  ws.run_forever()


def running3(btcusdt, ethusdt, bnbbtc):
  ws = websocket.WebSocketApp(socket3, on_message=on_message, on_close=on_close)
  ws.run_forever()


def ExpMovingAverage(values, window):
  weights = np.exp(np.linspace(-1., 0., window))
  weights /= weights.sum()
  a =  np.convolve(values, weights, mode='full')[:len(values)]
  a[:window] = a[window]
  return a.tolist()


processes = []

marker = int(input("Enter the number of values you want to get "))
window = int(input("Enter the window size for exponential moving average "))

if marker < window:
  window = int(input("Window size must be less than number of values "))

manager = multiprocessing.Manager()

btcusdt = manager.list()
ethusdt = manager.list()
bnbbtc = manager.list()
count = 1


p = multiprocessing.Process(target = running1, args=(btcusdt, ethusdt, bnbbtc))
p.start()
processes.append(p)
p = multiprocessing.Process(target = running2, args=(btcusdt, ethusdt, bnbbtc))
p.start()
processes.append(p)
p = multiprocessing.Process(target = running3, args=(btcusdt, ethusdt, bnbbtc))
p.start()
processes.append(p)


for p in processes:
    p.join()

ema_btcusdt = ExpMovingAverage(btcusdt, window)
ema_bnbbtc = ExpMovingAverage(bnbbtc, window)
ema_ethusdt = ExpMovingAverage(ethusdt, window)

currencies = [ema_btcusdt, ema_bnbbtc, ema_ethusdt]

currencies_name = ['EMA for BTC/USDT : ','EMA for BNB/BTC : ', 'EMA for ETH/USDT : ']
i = 0
ema_for_currencies = open("EMA_for_currencies.txt", "w+")
ema_for_currencies.write('Asked values : ' + str(marker) + '\n')
ema_for_currencies.write('Window size for EMA : ' + str(window) + '\n')
for currency in currencies:
  ema_for_currencies.write(currencies_name[i])
  i += 1
  for element in currency:
    ema_for_currencies.write(str(element) + ' ')
  ema_for_currencies.write('\n')
ema_for_currencies.close()

print('Done. Check the file')

#print('btcusdt vne ', btcusdt)
#print('bnbbtc vne ', bnbbtc)
#print('ethusdt vne ', ethusdt)

#print('ema for btcusdt ', ema_btcusdt)
#print('ema for bnbbtc ', ema_bnbbtc)
#print('ema for ethusdt ', ema_ethusdt)

Enter the number of values you want to get 7
Enter the window size for exponential moving average 3
Done. Check the file


In [24]:
import unittest

class UnitTest(unittest.TestCase):

  def test_1(self):
    self.assertEqual(ExpMovingAverage([10, 11, 12, 13] ,2), [11.268941421369995, 11.268941421369995, 11.268941421369995, 12.268941421369995])


  def test_2(self):
    self.assertEqual(ExpMovingAverage([1, 2, 3] , 1), [2.0, 2.0, 3.0])

  def test_3(self):
    self.assertEqual(ExpMovingAverage([0.3, 0.2, 0.8, 0.4, 0.1] , 2), [0.36136485282199704, 0.36136485282199704, 0.36136485282199704, 0.6924234314520019, 0.3193175735890015])

unittest.main(argv=[''], verbosity=2, exit=False)

test_1 (__main__.UnitTest) ... ok
test_2 (__main__.UnitTest) ... ok
test_3 (__main__.UnitTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK


In [22]:
running1(btcusdt, ethusdt, bnbbtc)